In [33]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [34]:
df=pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
df.shape

(891, 12)

In [36]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [37]:
df['family_size']=df['SibSp']+df['Parch']+1
df.drop(columns=['SibSp','Parch'],inplace=True)

In [38]:
df['individual_fare']=df['Fare']/(df['family_size'])
df.drop(columns=['Fare'],inplace=True)

In [39]:
def decide_FamilyType(x:int):
    if x==1:
        return "alone"
    elif x<5:
        return "small"
    else:
        return "large"
df['family_type']=df['family_size'].apply(decide_FamilyType)

In [40]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns='Survived'),df['Survived'],test_size=0.07,random_state=43)

In [41]:
trf1=ColumnTransformer(transformers=[
    ('impute_age',SimpleImputer(strategy='mean'),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[3])],
    remainder='passthrough',sparse_threshold=0.0)

In [42]:
x_train.head()

,Pclass,Sex,Age,Embarked,family_size,individual_fare,family_type
270,1,male,NaN,S,1,31.0000,alone
579,3,male,32.0,S,1,7.9250,alone
596,2,female,NaN,S,1,33.0000,alone
463,2,male,48.0,S,1,13.0000,alone
269,1,female,35.0,S,1,135.6333,alone


In [43]:
trf2=ColumnTransformer(transformers=[
    ('ohe_sex_embark_familyType',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,3,6])],
    remainder='passthrough')

In [44]:
from sklearn.ensemble import RandomForestClassifier
trf3=RandomForestClassifier(n_estimators=42,min_samples_split=12,max_features=4,max_samples=590,random_state=43)

In [45]:
pipe=make_pipeline(trf1,trf2,trf3)
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   sparse_threshold=0.0,
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [3])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embark_familyType',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3, 6])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_features=4, max_samples=590,
                                        min_samples_split=12, n_estimators=42,
                                        random_state=43))])

In [48]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean()

np.float64(0.8345454545454546)

In [50]:
from sklearn.metrics import accuracy_score
y_pred=pipe.predict(x_test)
accuracy_score(y_pred,y_test)

0.7936507936507936

In [49]:
import pickle
pickle.dump(pipe,open('model_pipe.pkl','wb'))